In [1]:
import ast
import glob
import json
import os
import re
import shutil

import albumentations as A
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from IPython.core.interactiveshell import InteractiveShell
from PIL import Image
from sklearn.model_selection import GroupKFold, KFold
from tqdm import tqdm

InteractiveShell.ast_node_interactivity = "all"
import seaborn as sns
import torch
import torchvision
from IPython.display import clear_output
from torchvision.ops import box_iou

In [2]:
TRAIN_DF_PART = "/app/_data/tensorflow-great-barrier-reef/train.csv"
IMAGE_FOLDER = "images"
LABEL_FOLDER = "labels"
SEED = 37

In [3]:
with open("/app/_data/sequences.json", "r") as f:
    seq_dict = json.load(f)

In [4]:
df = pd.read_csv(TRAIN_DF_PART)
df["img_path"] = (
    "/app/_data/tensorflow-great-barrier-reef/train_images/video_"
    + df.video_id.astype("str")
    + "/"
    + df.video_frame.astype("str")
    + ".jpg"
)
df["annotations"] = df["annotations"].apply(lambda x: ast.literal_eval(x))
df["len_annotation"] = df["annotations"].str.len()
df["image_id"] = df["image_id"].str.replace("-", "_", regex=True)
df["new_img_path"] = f"/app/_data/{IMAGE_FOLDER}/" + df["image_id"] + ".jpg"
df["label"] = df["len_annotation"].apply(lambda x: 0 if x == 0 else 1)
R = df[df["len_annotation"] == 0].shape[0] / df[df["len_annotation"] != 0].shape[0]

In [5]:
df["sequence"].unique().shape[0]

20

## KFold split

In [6]:
# seq_dict

In [7]:
# ids = '3'
# train_sequences = seq_dict[ids]['train']
# val_sequences = seq_dict[ids]['val']

In [8]:
# train = pd.concat(
#     [
#         df.query("sequence in @train_sequences and len_annotation!=0"),
#         df.query("sequence in @train_sequences and len_annotation==0").sample(
#             int(
#                 df.query("sequence in @train_sequences and len_annotation!=0").shape[0]
#                 * 0.07
#             )
#         ),
#     ]
# ).sample(frac = 1)
# val = pd.concat(
#     [
#         df.query("sequence in @val_sequences and len_annotation!=0"),
#         df.query("sequence in @val_sequences and len_annotation==0").sample(
#             int(
#                 df.query("sequence in @val_sequences and len_annotation!=0").shape[0]
#                 * R
#             )
#         ),
#     ]
# ).sample(frac = 1)

In [9]:
VIDEO_ID = 2
train = pd.concat(
    [
        df.query("video_id!=@VIDEO_ID and len_annotation!=0"),
        df.query("video_id!=@VIDEO_ID and len_annotation==0").sample(
            int(df.query("video_id!=@VIDEO_ID and len_annotation!=0").shape[0] * 0.03)
        ),
    ]
).sample(frac=1)
val = df.query("video_id==@VIDEO_ID").sample(frac=1)

In [10]:
train[["len_annotation", "label"]].sum()

len_annotation    9449
label             4242
dtype: int64

In [11]:
val[["len_annotation", "label"]].sum()

len_annotation    2449
label              677
dtype: int64

In [12]:
train[["len_annotation", "label"]].sum() / val[["len_annotation", "label"]].sum()

len_annotation    3.858310
label             6.265879
dtype: float64

In [13]:
train_ids = train.index.tolist()
val_ids = val.index.tolist()

len(train_ids), len(val_ids)

(4369, 8561)

In [14]:
train_txt = f"/app/_data/train_{SEED}_val{VIDEO_ID}.txt"
val_txt = train_txt.replace("train_", "val_")
data_yaml_path = f"/app/_data/yolov5_f2/data/reef_data_val{VIDEO_ID}.yaml"
data_yaml_path
val_txt

'/app/_data/yolov5_f2/data/reef_data_val2.yaml'

'/app/_data/val_37_val2.txt'

In [15]:
train_img_path = df.loc[train_ids, "new_img_path"].tolist()
val_img_path = df.loc[val_ids, "new_img_path"].tolist()
np.savetxt(
    train_txt,
    train_img_path,
    fmt="%s",
)
np.savetxt(val_txt, val_img_path, fmt="%s")

## Custimize parameters

In [16]:
from IPython.core.magic import register_line_cell_magic


@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, "w") as f:
        f.write(cell.format(**globals()))

In [17]:
%%writetemplate {data_yaml_path}

train: {train_txt}  # training directory
val: {val_txt}  # validation directory

# Classes
nc: 1  # number of classes
names: ["starfish"]  # class names

In [18]:
!cat {data_yaml_path}


train: /app/_data/train_37_val2.txt # training directory
val: /app/_data/val_37_val2.txt # validation directory

# Classes
nc: 1  # number of classes
names: ['starfish']  # class names


In [19]:
!cat /app/_data/yolov5_f2/data/hyps/hyp.scratch.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (effic

In [20]:
%%writetemplate /app/_data/yolov5_f2/data/hyps/hyp.custom.val2.yaml
# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.005  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.4  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 1.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.1  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.1  # image flip up-down (probability)
fliplr: 0.6  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.0  # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)

# yolov5 requirements and wandb

In [21]:
!pip install --upgrade wandb
clear_output()
import wandb

wandb.login()

wandb: Currently logged in as: tatanko (use `wandb login --relogin` to force relogin)


True

In [22]:
%cd /app/_data/yolov5_f2/
!pip install -r requirements.txt
clear_output()

In [23]:
# for path in glob.glob("/app/_data/yolov5_f2/runs/train/*"):
#     if 'NAME' in path:
#         path
#         shutil.rmtree(path)

# Train

In [24]:
WEIGHTS = "yolov5l6.pt"
IMG_SIZE = 1280 * 2
NAME = f"{WEIGHTS[:-3]}_{IMG_SIZE}_val{VIDEO_ID}_f2"
NAME

'yolov5l6_2560_val2_f2'

In [25]:
NAME

'yolov5l6_2560_val2_f2'

In [26]:
for path in glob.glob(f"/app/_data/yolov5_f2/*/*/{NAME}*"):
    shutil.rmtree(path)

In [27]:
!python train.py --img {IMG_SIZE} \
                --batch 4\
                --epochs 80 \
                --data {data_yaml_path} \
                --weights {WEIGHTS} \
                --name {NAME} \
                --hyp data/hyps/hyp.custom.val2.yaml \
                --single-cls \
                --patience 10 \
                --rect \
                --workers 0

wandb: Currently logged in as: tatanko (use `wandb login --relogin` to force relogin)
train: weights=yolov5l6.pt, cfg=, data=/app/_data/yolov5_f2/data/reef_data_val2.yaml, hyp=data/hyps/hyp.custom.val2.yaml, epochs=80, batch_size=4, imgsz=2560, rect=True, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=True, optimizer=SGD, sync_bn=False, workers=0, project=runs/train, name=yolov5l6_2560_val2_f2, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=10, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (Docker image), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 v6.0-193-gdb1f83b torch 1.9.1+cu111 CUDA:0 (NVIDIA GeForce RTX 3090, 24265MiB)

hyperparameters: lr0=0.005, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, w

In [28]:
def tp_fp_fn(gt, prediction, conf_thr):
    ious = np.arange(0.3, 0.81, 0.05)
    TP, FP, FN = (
        np.zeros(ious.shape[0], "int16"),
        np.zeros(ious.shape[0], "int16"),
        np.zeros(ious.shape[0], "int16"),
    )
    prediction = prediction[prediction[:, 4] > conf_thr]
    bboxes = prediction[:, :4].astype("int")
    bboxes[:, 0] = bboxes[:, 0] - bboxes[:, 2] / 2
    bboxes[:, 1] = bboxes[:, 1] - bboxes[:, 3] / 2
    bboxes[:, 2] = bboxes[:, 0] + bboxes[:, 2]
    bboxes[:, 3] = bboxes[:, 1] + bboxes[:, 3]
    if bboxes.size != 0:
        if gt.size == 0:
            fp = bboxes.shape[0]
            FP = np.full(ious.shape[0], fp, "int16")
        else:
            iou_matrix = box_iou(torch.Tensor(gt), torch.Tensor(bboxes))
            for n, iou_thr in enumerate(ious):
                x = torch.where(iou_matrix >= iou_thr)
                tp = np.unique(x[0]).shape[0]
                fp = bboxes.shape[0] - tp
                fn = gt.shape[0] - tp
                TP[n] = tp
                FP[n] = fp
                FN[n] = fn
    else:
        if gt.size != 0:
            fn = gt.shape[0]
            FN = np.full(ious.shape[0], fn, "int16")
    return TP, FP, FN

In [6]:
with open("/app/f2_results.json", "r") as f:
    res_dict = json.load(f)

In [137]:
conf_thres = np.arange(0.1, 0.61, 0.01)
ious = np.arange(0.3, 0.81, 0.05)
res = np.zeros([conf_thres.shape[0], 3, ious.shape[0]])

# path = f"/app/_data/yolov5_f2/runs/train/{NAME}/weights/best.pt"
path = '/app/_data/yolov5_f2/runs/train/yolov5m6_2560_val2_f2/weights/best.pt'

IMG_SIZE = IMG_SIZE
model = torch.hub.load(
    "/app/_data/yolov5", "custom", path=path, source="local", force_reload=True
)
model.conf = 0.01
# chose validation set
df_test = val.copy()
# computing f2 score
for ix in tqdm(df_test.index.tolist()):
    img = np.array(Image.open(df_test.loc[ix, "img_path"]))
    prediction = model(img, size=IMG_SIZE, augment=True).xywh[0].cpu().numpy()
    prediction = prediction[prediction[:, 4] > 0.1]
    gt = np.array([list(x.values()) for x in df_test.loc[ix, "annotations"]])
    if gt.size:
        gt[:, 2] = gt[:, 2] + gt[:, 0]
        gt[:, 3] = gt[:, 3] + gt[:, 1]
    for n, c_th in enumerate(conf_thres):
        TP, FP, FN = tp_fp_fn(gt, prediction, c_th)
        res[n, 0, :] += TP
        res[n, 1, :] += FP
        res[n, 2, :] += FN

YOLOv5 🚀 v6.0-193-gdb1f83b torch 1.9.1+cu111 CUDA:0 (NVIDIA GeForce RTX 3090, 24265MiB)

Fusing layers... 
Model Summary: 378 layers, 35248920 parameters, 0 gradients, 49.0 GFLOPs
Adding AutoShape... 
100% 8561/8561 [18:37<00:00,  7.66it/s]


In [138]:
F2 = np.zeros(conf_thres.shape[0])
for c in range(conf_thres.shape[0]):
    TP = res[c, 0, :]
    FP = res[c, 1, :]
    FN = res[c, 2, :]
    recall = TP / (TP + FN)
    precission = TP / (TP + FP)
    f2 = 5 * precission * recall / (4 * precission + recall + 1e-16)
    F2[c] = np.mean(f2)
if path not in res_dict:
    res_dict[path] = {
        IMG_SIZE: {
            "best": [
                np.round(conf_thres[np.argmax(F2)], 2),
                np.round(np.max(F2), 4),
            ],
            "all": list(np.round(F2, 4)),
        }
    }
else:
    res_dict[path][IMG_SIZE] = {
        "best": [
            np.round(conf_thres[np.argmax(F2)], 2),
            np.round(np.max(F2), 4),
        ],
        "all": list(np.round(F2, 4)),
    }

In [139]:
res_dict[path][IMG_SIZE]

{'best': [0.23, 0.659],
 'all': [0.6293,
  0.6344,
  0.6389,
  0.643,
  0.6462,
  0.6486,
  0.6517,
  0.6531,
  0.654,
  0.6545,
  0.6557,
  0.6576,
  0.6583,
  0.659,
  0.6586,
  0.659,
  0.6586,
  0.658,
  0.6579,
  0.6567,
  0.6559,
  0.6556,
  0.6554,
  0.6553,
  0.6551,
  0.6531,
  0.6528,
  0.6526,
  0.6516,
  0.6496,
  0.6475,
  0.6463,
  0.6464,
  0.6456,
  0.6432,
  0.6401,
  0.6399,
  0.6363,
  0.635,
  0.6342,
  0.6295,
  0.627,
  0.6259,
  0.6247,
  0.6217,
  0.6191,
  0.6166,
  0.6133,
  0.6103,
  0.6088,
  0.6058]}

In [97]:
def show_img(img, bbox, pred=None):
    img_h, img_w = img.shape[:2]

    fig, ax = plt.subplots(1, 1, figsize=(15, 8))
    ax.imshow(img)
    for i in range(len(bbox)):
        x_c, y_c, w, h = bbox[i]
        rect = plt.Rectangle(
            [(x_c - w / 2) * img_w, (y_c - h / 2) * img_h],
            w * img_w,
            h * img_h,
            ec="b",
            fc="none",
            lw=2.0,
        )
        ax.add_patch(rect)
    if pred is not None:
        for i in range(len(pred)):
            x_c, y_c, w, h = pred[i]
            rect = plt.Rectangle(
                [x_c - w / 2, y_c - h / 2],
                w,
                h,
                ec="r",
                fc="none",
                lw=2.0,
            )
            ax.add_patch(rect)
    plt.show();

In [140]:
with open("/app/f2_results.json", "w") as f:
    json.dump(res_dict, f)

In [142]:
transform = A.Compose(
    [
#         A.ShiftScaleRotate(shift_limit=0.05, scale_limit=[0.05, 0.1], rotate_limit=10, border_mode=0, value=(114, 114, 114),p=1.0),
        A.HueSaturationValue(
            hue_shift_limit=3, sat_shift_limit=3, val_shift_limit=5, p=1
        ),
        A.RandomBrightnessContrast(
            brightness_limit=0.01, contrast_limit=0.05, brightness_by_max=True, p=1
        ),
        A.Flip(p=1)
    ],
    bbox_params=A.BboxParams(
        format="yolo", min_visibility=0.4, label_fields=["class_labels"]
    ),
)

In [143]:
conf_thres = np.arange(0.1, 0.61, 0.01)
ious = np.arange(0.3, 0.81, 0.05)
res = np.zeros([conf_thres.shape[0], 3, ious.shape[0]])
path = f"/app/_data/yolov5_f2/runs/train/{NAME}/weights/best.pt"
IMG_SIZE = IMG_SIZE
model = torch.hub.load(
    "/app/_data/yolov5", "custom", path=path, source="local", force_reload=True
)
model.conf = 0.01
df_test = val.copy()

YOLOv5 🚀 v6.0-193-gdb1f83b torch 1.9.1+cu111 CUDA:0 (NVIDIA GeForce RTX 3090, 24265MiB)

Fusing layers... 
Model Summary: 476 layers, 76118664 parameters, 0 gradients, 110.0 GFLOPs
Adding AutoShape... 


In [144]:
df_test = pd.concat([val.query("len_annotation>0"),
           val.query("len_annotation==0").sample(1000)]).sample(frac=1)

In [10]:
res_dict['/app/_data/yolov5_f2/runs/train/yolov5m6_2560_val2_f2/weights/best.pt']

{'2560': {'best': [0.23, 0.659],
  'all': [0.6293,
   0.6344,
   0.6389,
   0.643,
   0.6462,
   0.6486,
   0.6517,
   0.6531,
   0.654,
   0.6545,
   0.6557,
   0.6576,
   0.6583,
   0.659,
   0.6586,
   0.659,
   0.6586,
   0.658,
   0.6579,
   0.6567,
   0.6559,
   0.6556,
   0.6554,
   0.6553,
   0.6551,
   0.6531,
   0.6528,
   0.6526,
   0.6516,
   0.6496,
   0.6475,
   0.6463,
   0.6464,
   0.6456,
   0.6432,
   0.6401,
   0.6399,
   0.6363,
   0.635,
   0.6342,
   0.6295,
   0.627,
   0.6259,
   0.6247,
   0.6217,
   0.6191,
   0.6166,
   0.6133,
   0.6103,
   0.6088,
   0.6058]}}

In [1]:
# img_w = 1280
# img_h = 720
# for ix in tqdm(df_test.index):
#     image_path = df_test.loc[ix, "img_path"]
#     img_name = df_test.loc[ix, "image_id"]
#     annotations = df_test.loc[ix, "annotations"]
#     img = np.array(Image.open(image_path))
#     bboxes = np.zeros([len(annotations), 5])
#     for i in range(len(annotations)):
#         xmin = annotations[i]["x"] / img_w
#         ymin = annotations[i]["y"] / img_h
#         width = annotations[i]["width"] / img_w
#         height = annotations[i]["height"] / img_h
#         width = width if (width + xmin) <= 1 else (1 - xmin)
#         height = height if (height + ymin) <= 1 else (1 - ymin)
#         x_center = xmin + width / 2
#         y_center = ymin + height / 2
#         bboxes[i:, 0] = 0
#         bboxes[i:, 1] = x_center
#         bboxes[i:, 2] = y_center
#         bboxes[i:, 3] = width
#         bboxes[i:, 4] = height
#     for n in range(5):
#         transformed = transform(
#             image=img,
#             bboxes=bboxes[:, 1:],
#             class_labels=bboxes[:, 0],
#         )
#         a_img, a_bbox = transformed["image"], transformed["bboxes"]
#         prediction = model(a_img, size=IMG_SIZE, augment=True).xywh[0].cpu().numpy()
#         prediction = prediction[prediction[:, 4] > 0.1]
#         gt = np.array(a_bbox)
#         if gt.size:
#             gt[:, 0] *= 1280
#             gt[:, 1] *= 720
#             gt[:, 2] *= 1280
#             gt[:, 3] *= 720
#             gt[:, 0] = gt[:, 0] - gt[:, 2] / 2
#             gt[:, 1] = gt[:, 1] - gt[:, 3] / 2
#             gt[:, 2] = gt[:, 0] + gt[:, 2]
#             gt[:, 3] = gt[:, 1] + gt[:, 3]

#         for n, c_th in enumerate(conf_thres):
#             TP, FP, FN = tp_fp_fn(gt, prediction, c_th)
#             res[n, 0, :] += TP
#             res[n, 1, :] += FP
#             res[n, 2, :] += FN
#         show_img(a_img, a_bbox, prediction[:, :4])

In [146]:
prediction

array([], shape=(0, 6), dtype=float32)

In [152]:
F2 = np.zeros(conf_thres.shape[0])
for c in range(conf_thres.shape[0]):
    TP = res[c, 0, :]
    FP = res[c, 1, :]
    FN = res[c, 2, :]
    recall = TP / (TP + FN)
    precission = TP / (TP + FP)
    f2 = 5 * precission * recall / (4 * precission + recall + 1e-16)
    F2[c] = np.mean(f2)
print("best", np.round(conf_thres[np.argmax(F2)], 2), np.round(np.max(F2), 4), "\nall", list(np.round(F2, 4)),)

best 0.18 0.6667 
all [0.6612, 0.6612, 0.6612, 0.6612, 0.6612, 0.6612, 0.6639, 0.6639, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667]


In [133]:
'/app/_data/yolov5_f2/runs/train/yolov5l6_2560_val2_f2/weights/best.pt'


'/app/_data/yolov5_f2/runs/train/yolov5l6_2560_val2_f2/weights/best.pt'

In [141]:
shutil.copy(path, f'/app/_data/YOLOv5full_train_weights/yolov5m6_2560_val2_f2.pt')

'/app/_data/YOLOv5full_train_weights/yolov5m6_2560_val2_f2.pt'

In [135]:
NAME

'yolov5l6_2560_val2_f2'